# RemNote Explorer

Exploration of the raw RemNote export. Extract your export into the `export` folder.
There should be two files: `export/cards.json` (spaced repetition related) and `export/rem.json` (all your rems).

In [37]:
import json
from pprint import pprint, pp

rem = json.load(open('export/rem.json'))
cards = json.load(open('export/cards.json'))
pp(rem, depth=1)
pp(cards, depth=1)

{'exportDate': '2020-08-13T20:47:56.498Z', 'exportVersion': 1, 'docs': [...]}
{'exportDate': '2020-08-13T20:47:56.575Z', 'exportVersion': 1, 'docs': [...]}


In [38]:
print('Number of cards:', len(cards['docs']))
base = rem['docs']
print('Number of rem: ', len(base))

Number of cards: 265
Number of rem:  740


These keys are used in the export

In [22]:
keys = set()
for rem in base:
    keys.update(rem.keys())
keys

{'_id',
 'article',
 'autoImport',
 'backSpacedRepetition',
 'children',
 'cp',
 'createdAt',
 'crt',
 'docUpdated',
 'dontSync',
 'embeddedSearchId',
 'enableBackSR',
 'excludedTypeParents',
 'folderOpen',
 'forget',
 'hasImplicitTag',
 'history',
 'isListItem',
 'key',
 'mc',
 'noBullet',
 'owner',
 'p',
 'parent',
 'portalType',
 'portalsIn',
 'preMigrateChildren',
 'preMigrateSubBlocks',
 'preMigrationTypeChildren',
 'rcre',
 'rcrp',
 'rcrs',
 'rcrt',
 'references',
 'rsp',
 's',
 'searchAliases',
 'searchResults',
 'selectedInSearch',
 'spacedRepetition',
 'srcRemC',
 'srcRemId',
 'subBlocks',
 'tcsp',
 'timesMovedTo',
 'type',
 'typeChildren',
 'typeParents',
 'u',
 'value',
 'valueType'}

## Top-level Rem

In [39]:
toplevel = [r for r in base if 'parent' not in r or not r['parent']]
len(toplevel)

118

In [58]:
def rem_by_id(id, base):
    # TODO: Speed up with index
    for r in base:
        if r['_id'] == id:
            return r
    raise FileNotFoundError(f'Rem with id {id} not found')

def remtext(rem, base: list):
    """Print the rem title/name/text/key.

    This is written in the `key` field of a rem which is a list
    of words. There are some special cases:
    
    - Rem references are embedded as a dict with an `_id`.
      Example: `{'_id': 'Muw33mm7WgmyyHPCY', 'i': 'q'}`
    TODO: What is `'i': 'q'`?

    - Example: `{'text': ' \\sum_{i=0}^n i ', 'i': 'm', 'x': True}`

    TODO: Why are Words sometimes split on spaces like
        ['Building', ' ', 'A', ' ', 'Second', ' ', 'Brain']
    """

    # replace 
    rem_text = rem['key']

    def is_reference(part):
        return type(part) is dict and '_id' in part
    def is_text(part):
        return type(part) is dict and 'text' in part

    def plain(part):
        if is_reference(part):
            ref_id = part['_id']
            rem = rem_by_id(ref_id, base)
            return remtext(rem, base)
        elif is_text(part):
            return f'$${part["text"]}$$'
        assert type(part) is str
        return part

    plain = [plain(part) for part in rem_text]
    return ''.join(plain)

    

In [64]:
for r in toplevel[:10]:
    print(f'{remtext(r, base):40.40}', r['key'])

Experiment                               ['Experiment']
Valence Electron                         ['Valence Electron']
Chemical Bond                            ['Chemical Bond']
Noun                                     ['Noun']
Static Array                             ['Static Array']
Selection Sort                           ['Selection Sort']
Computer                                 ['Computer']
Keyboard Shortcut                        ['Keyboard Shortcut']
Learning to Learn                        ['Learning to Learn']
In Place Sorting Algorithm               ['In Place ', {'_id': 'Muw33mm7WgmyyHPCY', 'i': 'q'}]
